Kubeless
========

Kubeless ist ein natives serverloses Framework, mit dem Sie kleine Code-Bits (Funktionen) bereitstellen können, ohne sich um die zugrunde liegende Infrastruktur kümmern zu müssen. Es wurde entwickelt, um auf einem Kubernetes-Cluster eingesetzt zu werden und alle großen Kubernetes-Primitive zu nutzen.

Installation
----------------

Zuerst brauchen wir das kubeless CLI. Diese können wir von der [Release Page](https://github.com/kubeless/kubeless/releases) downloaden, entpacken und hier abstellen.

**Installation Serverteil:**

In [ ]:
%%bash
export RELEASE=$(curl -s https://api.github.com/repos/kubeless/kubeless/releases/latest | grep tag_name | cut -d '"' -f 4)
kubectl create ns kubeless
kubectl apply -f https://github.com/kubeless/kubeless/releases/download/$RELEASE/kubeless-$RELEASE.yaml
echo "----------------------------------------"
kubectl get all -n kubeless

**Installation CLI:**

In [ ]:
%%bash
export RELEASE=$(curl -s https://api.github.com/repos/kubeless/kubeless/releases/latest | grep tag_name | cut -d '"' -f 4)
curl -OL https://github.com/kubeless/kubeless/releases/download/$RELEASE/kubeless_linux-amd64.zip && \
  unzip kubeless_linux-amd64.zip && \
  mv bundles/kubeless_linux-amd64/kubeless kubeless

- - -
Nun sind wir bereit und können sogenannte **Functions** deployen.

Die Funktion bzw. Datei erstellen wir uns mittels `cat` und veröffentlichen diese mit `kubeless`

In [ ]:
%%bash
# Create Function (bzw. Datei helloget.js)
cat <<%EOF% >test.py
def hello(event, context):
  print event
  return event['data']
%EOF%

# Deploy Function
./kubeless function deploy hello --runtime python2.7 \
                                 --from-file test.py \
                                 --handler test.hello

Ob die Funktion veröffentlich wurde und aktiv ist, kann via `kubeless` abgefragt werden.

Welche Ressourcen in Kubernetes angelegt wurden liefert `kubectl get all -l function=<name>`

In [ ]:
! ./kubeless function ls
! kubectl get all -l function=hello

Nach erfolgreicher veröffentlichung der Funktion können wir diese nun aufrufen:

In [ ]:
! ./kubeless function call hello --data 'Hello world!'

- - -
Was fehlt ist ein User Interface um unsere Funktionen zu verwalten. Glücklicherweise liefert `kubeless` ein solches gleich mit:

In [ ]:
%%bash
kubectl apply -f https://raw.githubusercontent.com/kubeless/kubeless-ui/master/k8s.yaml
kubectl config view -o=jsonpath='{ .clusters[0].cluster.server }' | \
sed -e 's/https:/http:/' -e "s/6443/$(kubectl get service -l controller=ui --namespace kubeless -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')/"

- - -
Aufräumen

In [ ]:
! ./kubeless function delete hello